In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
# Load movie data
import pandas as pd
movie_df = pd.read_pickle('../dump/movie_data')

In [12]:
movie_trimmed = movie_df.drop(columns=['rating','vote','metascore','keywords',\
                                       'link_d','link_s','link_w'])
movie_trimmed.rename(columns = {'certificate':'MPAA'},inplace=True)

In [13]:
movie_trimmed

,movie_title,MPAA,genre,release_date,budget,opening_weekend_usa,gross_usa,gross_world,runtime,director,writer,star,distributor,language,country
0,After We Collided,R,"[Drama, Romance]",2020-10-23,NaN,422899.0,2386483.0,4.799041e+07,105.0,Roger Kumble,"[Anna Todd, Mario Celaya]","[Josephine Langford, Hero Fiennes Tiffin, Dyla...",CalMaple,"[English, Chinese]",[USA]
1,The Godfather,R,"[Crime, Drama]",1972-03-24,6000000.0,302393.0,134966411.0,2.461210e+08,175.0,Francis Ford Coppola,"[Mario Puzo, Francis Ford Coppola]","[Marlon Brando, Al Pacino, James Caan]",ParamountPictures,"[English, Italian, Latin]",[USA]
2,Harry Potter and the Sorcerer's Stone,PG,"[Adventure, Family, Fantasy]",2001-11-16,125000000.0,90294621.0,318087620.0,1.006918e+09,159.0,Chris Columbus,"[J.K. Rowling, Steve Kloves]","[Daniel Radcliffe, Rupert Grint, Richard Harris]",WarnerBros.,[English],"[UK, USA]"
3,Unknown,PG-13,"[Action, Thriller]",2011-02-18,30000000.0,21856389.0,63686397.0,1.357100e+08,113.0,Jaume Collet-Serra,"[Oliver Butcher, Stephen Cornwell]","[Liam Neeson, Diane Kruger, January Jones]",DarkCastleEntertainment,"[English, German, Turkish, Arabic]","[UK, Germany, France, USA]"
4,The Lord of the Rings: The Fellowship of the Ring,PG-13,"[Action, Adventure, Drama]",2001-12-19,93000000.0,47211490.0,315544750.0,8.881591e+08,171.0,Peter Jackson,"[J.R.R. Tolkien, Fran Walsh, 2 more credits]","[Elijah Wood, Ian McKellen, Orlando Bloom]",NewLineCinema,"[English, Sindarin]","[NewZealand, USA]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,From the Terrace,Passed,"[Drama, Romance]",1960-09-09,NaN,NaN,11336000.0,NaN,149.0,Mark Robson,"[John O'Hara, Ernest Lehman]","[Paul Newman, Joanne Woodward, Myrna Loy]",Linebrook,[English],[USA]
1996,V.I. Warshawski,R,"[Action, Comedy, Crime]",1991-07-26,NaN,3603338.0,11128309.0,1.112831e+07,89.0,Jeff Kanew,"[Sara Paretsky, Edward Taylor, 3 more credits]","[Kathleen Turner, Jay O. Sanders, Charles Durn...",HollywoodPictures,[English],[USA]
1997,Anne of Green Gables,Passed,"[Comedy, Drama, Family]",1934-11-23,226000.0,NaN,NaN,NaN,78.0,George Nichols Jr.,"[Sam Mintz, Lucy Maud Montgomery]","[Anne Shirley, Tom Brown, O.P. Heggie]",RKORadioPictures,[English],[USA]
1998,Odds Against Tomorrow,Approved,"[Crime, Drama, Thriller]",1959-11-01,NaN,NaN,NaN,NaN,100.0,Robert Wise,"[William P. McGivern, Abraham Polonsky, 2 more...","[Harry Belafonte, Robert Ryan, Gloria Grahame]",HarBelProductions,[English],[USA]


In [18]:
movie_trimmed.dropna(inplace=True)

In [19]:
movie_trimmed.shape

(960, 15)

In [20]:
movie_trimmed.MPAA.unique()

array(['R', 'PG', 'PG-13', 'G', 'Passed', 'NC-17', 'Approved', 'X',
       'M/PG', 'Not'], dtype=object)

In [25]:
movie_trimmed.MPAA.value_counts()

R           434
PG-13       301
PG          179
G            36
Passed        4
Not           2
X             1
M/PG          1
Approved      1
NC-17         1
Name: MPAA, dtype: int64

In [31]:
movie_trimmed.MPAA.replace(['G','Passed','Not','X','M/PG','Approved','NC-17'],'Other',inplace=True)

In [32]:
movie_trimmed.MPAA.value_counts()

R        434
PG-13    301
PG       179
Other     46
Name: MPAA, dtype: int64

In [34]:
movie_trimmed.columns

Index(['movie_title', 'MPAA', 'genre', 'release_date', 'budget',
       'opening_weekend_usa', 'gross_usa', 'gross_world', 'runtime',
       'director', 'writer', 'star', 'distributor', 'language', 'country'],
      dtype='object')

In [38]:
# Train test split
from sklearn.model_selection import train_test_split

X= movie_trimmed[['movie_title', 'MPAA', 'genre', 'release_date', 'budget','runtime',
       'director', 'writer', 'star', 'distributor', 'language', 'country']]

y = movie_trimmed['opening_weekend_usa']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [39]:
# Preprocessing
# Categorical varibale: MPAA
cat_variables = ['MPAA']
X_train_cat = X_train[cat_variables]

In [41]:
# Instantiate a OneHotEncoder object
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, drop='first')
ohe.fit(X_train_cat)
cats = ohe.transform(X_train_cat)

In [43]:
# Create the categorical dataframe
columns = ohe.get_feature_names(cat_variables)
X_train_cat_df = pd.DataFrame(cats, columns = columns, index = X_train_cat.index)
X_train_cat_df.head()

,MPAA_PG,MPAA_PG-13,MPAA_R
230,0.0,0.0,1.0
22,0.0,0.0,1.0
1226,0.0,0.0,1.0
1326,0.0,0.0,1.0
501,0.0,1.0,0.0


In [ ]:
# Repeat transformation for testing set

### OneHotEncoder with list item (e.g. genre)

In [158]:
# Example
d = {'A': [[5,7], [3, 4, 5], [2], [1,2,3,4]]}
df = pd.DataFrame(data=d)
df

,A
0,"[5, 7]"
1,"[3, 4, 5]"
2,[2]
3,"[1, 2, 3, 4]"


In [91]:
# Use MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
df_ohe = pd.DataFrame(mlb.fit_transform(df['A']),columns = mlb.classes_, index=df.index)
df_ohe

,1,2,3,4,5,7
0,0,0,0,0,1,1
1,0,0,1,1,1,0
2,0,1,0,0,0,0
3,1,1,1,1,0,0


In [195]:
# Test on genre
cat_variable = ['genre']
X_train_cat = X_train[cat_variable]

In [199]:
X_train_cat

,genre
230,"[Drama, History]"
22,"[Crime, Drama, Mystery]"
1226,"[Crime, Drama, Thriller]"
1326,[Drama]
501,"[Drama, Fantasy, Mystery]"
...,...
1361,"[Action, Biography, Western]"
402,"[Animation, Drama, Family]"
436,"[Action, Adventure, Fantasy]"
901,"[Crime, Drama, Mystery]"


In [173]:
X_train_cat_test = X_train_cat.iloc[:3,:].copy()
X_train_cat_test

,genre
230,"[Drama, History]"
22,"[Crime, Drama, Mystery]"
1226,"[Crime, Drama, Thriller]"


In [243]:
X_train_cat_set = X_train_cat.copy()

In [144]:
def list_set(my_list):
    return set(my_list)

In [254]:
# Keep only the top 10 genre
genre_list = ['Drama','Adventure','Comedy','Action','Crime','Romance','Thriller',\
              'Mystery','Family','Fantasy']

In [255]:
X_train_cat_set['genre_clean']= X_train_cat_set.genre.apply(lambda x:\
                            [genre if genre in genre_list else 'Other'for genre in x])

In [256]:
X_train_cat_set

,genre,genre_clean
230,"[Drama, History]","[Drama, Other]"
22,"[Crime, Drama, Mystery]","[Crime, Drama, Mystery]"
1226,"[Crime, Drama, Thriller]","[Crime, Drama, Thriller]"
1326,[Drama],[Drama]
501,"[Drama, Fantasy, Mystery]","[Drama, Fantasy, Mystery]"
...,...,...
1361,"[Action, Biography, Western]","[Action, Other, Other]"
402,"[Animation, Drama, Family]","[Other, Drama, Family]"
436,"[Action, Adventure, Fantasy]","[Action, Adventure, Fantasy]"
901,"[Crime, Drama, Mystery]","[Crime, Drama, Mystery]"


In [259]:
# Convert genre column from list to set
X_train_cat_set['genre_clean'] = X_train_cat_set.genre_clean.apply(list_set)
X_train_cat_set[['genre_clean']]

,genre_clean
230,"{Other, Drama}"
22,"{Mystery, Crime, Drama}"
1226,"{Crime, Thriller, Drama}"
1326,{Drama}
501,"{Mystery, Drama, Fantasy}"
...,...
1361,"{Other, Action}"
402,"{Drama, Other, Family}"
436,"{Fantasy, Adventure, Action}"
901,"{Mystery, Crime, Drama}"


In [260]:
X_train_cat_df = pd.DataFrame(mlb.fit_transform(X_train_cat_set['genre_clean']),\
                              columns=mlb.classes_,index=X_train_cat_set.index)
X_train_cat_df

,Action,Adventure,Comedy,Crime,Drama,Family,Fantasy,Mystery,Other,Romance,Thriller
230,0,0,0,0,1,0,0,0,1,0,0
22,0,0,0,1,1,0,0,1,0,0,0
1226,0,0,0,1,1,0,0,0,0,0,1
1326,0,0,0,0,1,0,0,0,0,0,0
501,0,0,0,0,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1361,1,0,0,0,0,0,0,0,1,0,0
402,0,0,0,0,1,1,0,0,1,0,0
436,1,1,0,0,0,0,1,0,0,0,0
901,0,0,0,1,1,0,0,1,0,0,0


In [262]:
len(mlb.classes_),mlb.classes_

(11,
 array(['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Family',
        'Fantasy', 'Mystery', 'Other', 'Romance', 'Thriller'], dtype=object))

In [263]:
X_train_cat_df.sum(axis=0).sort_values(ascending=False)

Drama        491
Other        224
Adventure    191
Comedy       188
Action       175
Crime        155
Romance      152
Thriller     137
Mystery       98
Family        98
Fantasy       92
dtype: int64

In [218]:
X_train_cat_set.genre.replace(genre_list,'Other')

230                 {History, Drama}
22           {Mystery, Crime, Drama}
1226        {Crime, Thriller, Drama}
1326                         {Drama}
501        {Mystery, Drama, Fantasy}
                    ...             
1361    {Western, Biography, Action}
402       {Animation, Drama, Family}
436     {Fantasy, Adventure, Action}
901          {Mystery, Crime, Drama}
171                 {Western, Drama}
Name: genre, Length: 768, dtype: object

In [240]:
def genre_clean(genre_list):
    clean_genre = [x if x in genre_list else 'Other' for x in my_list]
    return clean_genre

In [222]:
s = ['History', 'Drama']

In [223]:
s.replace('History','Other')

AttributeError: 'list' object has no attribute 'replace'

In [234]:
s1 = [x if x in genre_list else 'Other' for x in s]
s1

['Other', 'Drama']

In [230]:
l = [22, 13, 45, 50, 98, 69, 43, 44, 1]

In [231]:
[x+1 if x >= 45 else x+5 for x in l]

[27, 18, 46, 51, 99, 70, 48, 49, 6]